Need to comment this file, we're working with notebooks

## Imports

In [171]:
import pysindy as ps
from pysindy import PDELibrary, WeakPDELibrary
from pysindy.optimizers import STLSQ
from pysindy.differentiation import SpectralDerivative
import matplotlib.pyplot as plt
from PIL import Image
import os
import numpy as np

## Data selection

In [172]:
""" Features directory of the changemaps """
#features_dir = r"../../generations/changemap_generations/cropped_cropped/result_20_cut_1_png"
#features_dir = r"../../generations/changemap_generations/cropped_resized/result_20_cut_1_png"

""" Features directory of the generated registered by deformation (these images are .jpg)"""
#features_dir = r"C:\Users\ariel\PycharmProjects\MLDM_Project\generations\deformation_generations\registered\24"
#features_dir = r"../../generations/deformation_generations/deformations/24"

""" Features directory of the phase correlation-matched images """
features_dir = r"C:\Users\ariel\PycharmProjects\MLDM_Project\generations\deformation_generations\registered\24"

""" Resolution (if necessary) to resize """
res = 224

In [173]:
frames = []

filenames = sorted(os.listdir(features_dir))
print(filenames) # Checking they've been imported in the correct order
for filename in filenames:
    if filename.endswith(".jpg"):
        img = Image.open(os.path.join(features_dir, filename))
        width, height = img.size
        #img = img.resize((res, res))
        img = img.convert('L')
        img = np.asarray(img)
        img = img / 255
        frames.append(img)
        
res = np.shape(frames[0])[0]

['registered_01.jpg', 'registered_02.jpg', 'registered_03.jpg', 'registered_04.jpg', 'registered_05.jpg', 'registered_06.jpg', 'registered_07.jpg', 'registered_08.jpg', 'registered_09.jpg', 'registered_10.jpg', 'registered_11.jpg', 'registered_12.jpg', 'registered_13.jpg', 'registered_14.jpg', 'registered_15.jpg', 'registered_16.jpg', 'registered_17.jpg', 'registered_18.jpg', 'registered_19.jpg', 'registered_20.jpg', 'registered_21.jpg', 'registered_22.jpg', 'registered_23.jpg', 'registered_24.jpg', 'registered_25.jpg', 'registered_26.jpg', 'registered_27.jpg', 'registered_28.jpg', 'registered_29.jpg', 'registered_30.jpg', 'registered_31.jpg', 'registered_32.jpg', 'registered_33.jpg', 'registered_34.jpg', 'registered_35.jpg', 'registered_36.jpg', 'registered_37.jpg', 'registered_38.jpg', 'registered_39.jpg', 'registered_40.jpg', 'registered_41.jpg', 'registered_42.jpg', 'registered_43.jpg', 'registered_44.jpg', 'registered_45.jpg', 'registered_46.jpg', 'registered_47.jpg', 'registered_

## Reshaping data, creating spatial grid

In [174]:
#Creating spatial grid
features = np.stack(frames, axis=0)
features = np.reshape(features, (len(features), res, res, 1))
spatial_grid_x, spatial_grid_y = np.meshgrid(np.arange(res), np.arange(res))
spatial_grid = np.stack((spatial_grid_x, spatial_grid_y), axis=-1)
print("Spatial grid shape:", np.shape(spatial_grid))
print("Features shape:", np.shape(features))

Spatial grid shape: (224, 224, 2)
Features shape: (50, 224, 224, 1)


## Feature library

In [175]:
#Define feature library
feature_lib = ps.PDELibrary(
    library_functions=[lambda x: x, lambda x: x ** 2, lambda x: x ** 3],
    derivative_order=4,
    spatial_grid=spatial_grid,
    include_interaction=False,
    function_names=[lambda x: x,lambda x: x + x,lambda x: x + x + x],
    multiindices=[[0,2], [2,0], [4,0], [0,4]],
    differentiation_method=SpectralDerivative,
)

parameter_lib = ps.PDELibrary(
    library_functions=[lambda x: x],
    include_interaction=False,
    function_names=[lambda x: x],
    include_bias=True,
)

lib = ps.ParameterizedLibrary(
    parameter_library=parameter_lib,
    feature_library=feature_lib,
    num_parameters=2,
    num_features=1,
)

## Defining optimizer and model

In [176]:
opt = ps.STLSQ(threshold=0.01, alpha=0.01, normalize_columns=False, verbose=True)
model = ps.SINDy(feature_library=lib, optimizer=opt, feature_names=["u", "F", "t"])

## Defining x_dot

In [177]:
x = np.asarray(features)
x_dot = model.differentiate(x=x)
u = [0.22, 0.0003]
print("Shape of x_dot", np.shape(x_dot))

[50  1  1  1]
Shape of x_dot (50, 224, 224, 1)


## Fitting model

In [178]:
model.fit(x, x_dot=x_dot, u=u, multiple_trajectories=False)  # Check spectral_derivative.py for the adjustment of differentiation axis and timestep, also not automatic
model.print()
print()

[  1   1 224   1]
[  1 224   1   1]
[  1 224   1   1]
[  1   1 224   1]
[  1   1 224   1]
[  1 224   1   1]
[  1 224   1   1]
[  1   1 224   1]
SLTSQ
n_features_selected =  21
n_features =  21
n_targets =  1
n_samples =  2508800
x.shape =  (2508800, 21)
y.shape =  (2508800, 1)

 Iteration ... |y - Xw|^2 ...  a * |w|_2 ...      |w|_0 ... Total error: |y - Xw|^2 + a * |w|_2
         0 ... 3.9034e+00 ... 1.9928e-01 ...          7 ... 4.1027e+00
         1 ... 3.9034e+00 ... 2.0386e-01 ...          6 ... 4.1073e+00
         2 ... 3.9034e+00 ... 2.0659e-01 ...          6 ... 4.1100e+00
(u)' = -609535373079.132 1 u + 1028457063749.289 1 uu + 819361868255.797 1 uuu + 2770615332174.426 F u + -4674804835209.016 F uu + -3724372128448.236 F uuu


In [179]:
np.set_printoptions(precision=3, suppress=True)
weights = model.coefficients()
normalized_weights = weights / np.sum(weights)
print(normalized_weights)

[[ 0.139 -0.234 -0.187 -0.    -0.    -0.    -0.    -0.631  1.065  0.848
  -0.    -0.    -0.    -0.    -0.    -0.    -0.    -0.    -0.    -0.
  -0.   ]]
